In [1]:
import ssl
import requests
from bs4 import BeautifulSoup as bs
import json
from transformers import pipeline
import nltk

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Specify the certificate path (workaround)
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://bajajfinserv.in/personal-loan'
base_url = 'https://bajajfinserv.in'

In [3]:
def get_valid_url(url, base_url):
    if url.startswith('http'):
        return url
    elif url.startswith('/'):
        return f"{base_url}{url}"
    else:
        return f"{base_url}/{url}"

In [4]:
# Initialize the model for keyword generation
keyword_generator = pipeline("text-generation", model="Voicelab/vlt5-base-keywords")  # Adjust 'device' as needed

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', '

In [5]:
# Make the initial request
response = requests.get(url)
print(response.status_code)

200


In [6]:
html_content = response.content
soup = bs(html_content, 'html.parser')

In [7]:
# Extract the URLs of the pages you want to visit
page_urls = [get_valid_url(link.get('href'), base_url) for link in soup.find_all('a') if link.get('href')]
print(page_urls)

['https://bajajfinserv.in/', 'https://cms-assets.bajajfinserv.in/is/content/bajajfinance/bajaj-finance-personal-loan-general-loan-application-t-cpdf?scl=1&fmt=pdf', 'https://bajajfinserv.in/webform/personal-loan', 'https://bajajfinserv.in/personal-loan-for-wedding', 'https://bajajfinserv.in/personal-loan-for-home-expenses', 'https://bajajfinserv.in/personal-loan-for-higher-education', 'https://bajajfinserv.in/personal-loan-for-medical-expenses', 'https://bajajfinserv.in/what-makes-flexi-hybrid-loan-better-option', 'https://bajajfinserv.in/flexi-loans', 'https://bajajfinserv.in/personal-loan-processing-fees-and-interest-rates', 'https://bajajfinserv.in/personal-loan', 'https://bajajfinserv.in/personal-loan-features', 'https://bajajfinserv.in/personal-loan-emi-calculator', 'https://bajajfinserv.in/personal-loan-eligibility-and-documents', 'https://bajajfinserv.in/how-to-apply-for-personal-loan', 'https://bajajfinserv.in/personal-loan-processing-fees-and-interest-rates', 'https://bajajfin

In [8]:
# Use the keyword generation model to extract keywords
text_content = soup.get_text()
keywords = keyword_generator(text_content, max_length=50, num_return_sequences=1)[0]['generated_text'].split(',')
print("Keywords:", keywords)

Keywords: ['\n\n\n\n\nPersonal Loan - Apply for Instant Personal Loan Online Of Up To Rs. 40 Lakh | Bajaj Finance Limited\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\n\n\nPersonal Loan\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy proceeding', ' you agree to our Terms and Conditions\n\n\n\n\n\n\n\n\nHave an ongoing loan application?\nRESUME\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nA loan for all your goals\n\n\n\n\n\n\n\n\nWedding\n                  \n\n\n\n\n\n\n\nHome Expenses\n                  \n\n\n\n\n\n\n\nHigher Education\n                  \n\n\n\n\n\n\n\nMedical Expenses\n                  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n3 unique variants of our personal loan\n\n\n\n\n\n\n\n                                Flexi Term Loan\n                

In [9]:
# Store the initial link and its keywords in a dictionary
links_and_tags = {url: keywords}

# Store all links and redirects in a set to remove duplicates
all_links = set()

In [10]:
# Extract links from the initial page
links = [get_valid_url(link.get('href'), base_url) for link in soup.find_all('a') if link.get('href')]
all_links.update(links)

In [11]:
# Iterate over page URLs
for url1 in page_urls:
    if not url1.startswith('http'):
        print(f"Invalid URL: {url1}. Skipping...")
        continue

    response = requests.get(url1)
    if response.status_code == 200:
        page_soup = bs(response.text, 'html.parser')

        # Extract links from each page
        links = [get_valid_url(link.get('href'), base_url) for link in page_soup.find_all('a') if link.get('href')]
        all_links.update(links)

        # Use the keyword generation model to extract keywords for each page
        text_content = page_soup.get_text()
        keywords = keyword_generator(text_content, max_length=50, num_return_sequences=1)[0]['generated_text'].split(',')
        print(f"Keywords for {url1}: {keywords}")

        # Store the link and its keywords in the dictionary
        links_and_tags[url1] = keywords

    else:
        print(f"Failed to fetch URL: {url1}. Status code: {response.status_code}")

Keywords for https://bajajfinserv.in/: ['\n\n\n\n\nBajaj Finserv: Loans', ' Cards', ' Insurance', ' Investments', ' Payments and more\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoans\n\n\n\n\n\n\n\nUPI\n\n\n\n\n\n\n\nWallet\n\n\n\n\n\n\n\nPassbook\n\n\n\n\n\n\n\nMore\n\n\n\n\n\n\n\n\n\nEMI Network Card\n\n\n\n\n\n\n\nEMI Card\n\n\n\n\n\n\n\nBajaj Mall\n\n\n\n\n\n\n\nPartners\n\n\n\n\n\n\n\nOffers\n\n\n\n\n\n\n\n\nCo-brand Credit Card\n\n\n\n\n\n\n\nCheck Offer\n\n\n\n\n\n\n\nJoining Bonus\n\n\n\n\n\n\n\nOffers\n\n\n\n\n\n\n\nCard Benefits\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nRegister for Do Not Call service\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIntroducing Bajaj Finance Digital FD\nBajaj Finance launches Digital Fixed Deposit – a secure', ' hassle-free investment optio

: 

In [ ]:
# Filter links with only the base URL and without "javascript:void(0)" and "mailto"
filtered_links = [link for link in all_links if link.startswith(base_url) and "javascript:void(0)" not in link and "mailto" not in link]

In [ ]:
# Save links and associated keywords to a JSON file
json_filename = 'links_and_tags_keywords.json'
with open(json_filename, 'w') as json_file:
    json.dump(links_and_tags, json_file, indent=2)

print(f"Links and associated tags saved to {json_filename}.")